In [11]:
import pandas as pd
import numpy as np
from numpy import arctan as atan, sqrt, sin, cos
from scipy.stats import pearsonr

### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_v\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |

In [7]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data' / 'data'

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.0371,921.25,1,bed,M,d1p01M,1
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.8920,920.25,1,bed,M,d1p01M,1


Each row $i$ of data in the dataframe is treated as the trailing edge of a frame of data from $t_i-\delta t$ to $t_i$, within each frame.

In [12]:
def corr(x, y):
    if len(x) == len(y) == 1:
        return 0
    return pearsonr(x, y)

def get_frame(df, t, delta_t):
    frame_df = df[(df['time'] > t - delta_t) & (df['time'] <= t)]
    
    if not len(frame_df):
        return None

    t = frame_df['time']
    a_f = frame_df['front']
    a_l = frame_df['lateral']
    a_v = frame_df['vertical']

    theta = atan(a_f / sqrt(a_v**2 + a_l**2))
    alpha = atan(a_l / a_v)
    beta = atan(a_l / a_f)

    a_x = a_v * sin(theta) + a_f * cos(theta)
    a_y = a_v * sin(alpha) + a_l * cos(alpha)
    a_z = 1 + a_v * cos(theta) * cos(alpha) + a_l * sin(alpha) + a_f * sin(theta)
    
    
    frame = dict({
        't': t[0],
        'a_f': a_f[0],
        'a_l': a_l[0],
        'a_v': a_v[0],
        'theta': theta[0],
        'alpha': alpha[0],
        'beta': beta[0],
        'a_x': a_x[0],
        'a_y': a_y[0],
        'a_z': a_z[0],
        'mean_a_f': a_f.mean(),
        'mean_a_l': a_l.mean(),
        'mean_a_v': a_v.mean(),
        'max_a_f': a_f.max(),
        'max_a_l': a_l.max(),
        'max_a_v': a_v.max(),
        'min_a_f': a_f.min(),
        'min_a_l': a_l.min(),
        'min_a_v': a_v.min(),
        'coor_a_f-a_l': corr(a_f, a_l),
        'coor_a_f-a_v': corr(a_f, a_v),
        'coor_a_l-a_v': corr(a_l, a_v)
    })
    return pd.DataFrame(frame, index=frame_df.index[0:1])


In [13]:
participants = sensor_df['participant'].unique()
participant = participants[0]

participant_df = sensor_df[sensor_df['participant'] == participant]



In [14]:

delta_t = 2.0
t_freq = 0.5
min_t, max_t = participant_df['time'].min(), participant_df['time'].max()
t = min_t
while t <= max_t:
    frame_df = get_frame(participant_df, t, delta_t)
    
    t += t_freq
    break

ValueError: x and y must have length at least 2.

In [20]:
from queue import Queue

In [ ]:
class MessageQueue:
    
    def __init__(self):
        self.queue = Queue()
        
    

In [15]:

class RingBuffer:
    
    def __init__(self, size=8):
        self.size = size
        self.buffer = np.array(size)
        self.i = 0
        
    def write(self, value):
        self.buffer[self.i] = value
        self.i = (self.i + 1) % self.size()



In [27]:
delta_t = .5
time_slice = (participant_df['time'] // delta_t).astype(int)

In [28]:
time_slice

0        0
1        1
2        3
3        3
4        5
      ... 
396    502
397    502
398    503
399    506
400    508
Name: time, Length: 401, dtype: int64